In [2]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_csv('../data/processed/df_listo_para_segmentar.csv',sep=';')
df['CUT Comuna Destino'].unique()

array([ 1402,  1403,  1404,  1405,  2101,  2102,  2103,  2104,  2201,
        2203,  2301,  2302,  3101,  3102,  3103,  3201,  3202,  3301,
        3302,  3303,  3304,  4101,  4102,  4103,  4104,  4105,  4106,
        4201,  4202,  4203,  4204,  4301,  4302,  4303,  4304,  4305,
        5101,  5102,  5103,  5105,  5107,  5109,  5201,  5301,  5302,
        5303,  5304,  5401,  5402,  5403,  5404,  5405,  5501,  5502,
        5503,  5504,  5506,  5601,  5602,  5603,  5604,  5605,  5606,
        5701,  5703,  5704,  5705,  5706,  5801,  5802,  5803,  5804,
        6101,  6102,  6105,  6106,  6107,  6108,  6110,  6112,  6113,
        6114,  6115,  6116,  6117,  6201,  6205,  6301,  6303,  6305,
        6310,  7101,  7102,  7103,  7105,  7109,  7110,  7201,  7203,
        7301,  7303,  7304,  7305,  7306,  7307,  7308,  7401,  7402,
        7403,  7404,  7405,  7406,  7407,  7408,  8101,  8102,  8103,
        8105,  8106,  8107,  8108,  8110,  8111,  8112,  8201,  8202,
        8203,  8204,

In [3]:
st_comuna_destino = df[['CUT Comuna Destino','Anio','CUT Mes','Viajes Ocasionales']]
st_comuna_destino = st_comuna_destino[st_comuna_destino['CUT Comuna Destino']==13101]
st_comuna_destino = st_comuna_destino.sort_values(by=['Anio','CUT Mes'])
st_comuna_destino.duplicated().sum()


np.int64(8994)

In [4]:
df_agrupado = st_comuna_destino.groupby(['CUT Comuna Destino', 'Anio', 'CUT Mes'], as_index=False)['Viajes Ocasionales'].sum()
df_agrupado

,CUT Comuna Destino,Anio,CUT Mes,Viajes Ocasionales
0,13101,2019,1,71456
1,13101,2019,2,58086
2,13101,2019,3,61188
3,13101,2019,4,35853
4,13101,2019,5,53024
...,...,...,...,...
61,13101,2024,2,51068
62,13101,2024,3,72723
63,13101,2024,4,88250
64,13101,2024,5,64225


In [5]:
df_agrupado['Fecha'] = pd.to_datetime(df_agrupado[['Anio', 'CUT Mes']].astype(str).agg('-'.join, axis=1), format='%Y-%m')
df_agrupado.set_index('Fecha', inplace=True)
df_agrupado

,CUT Comuna Destino,Anio,CUT Mes,Viajes Ocasionales
Fecha,,,,
2019-01-01,13101,2019,1,71456
2019-02-01,13101,2019,2,58086
2019-03-01,13101,2019,3,61188
2019-04-01,13101,2019,4,35853
2019-05-01,13101,2019,5,53024
...,...,...,...,...
2024-02-01,13101,2024,2,51068
2024-03-01,13101,2024,3,72723
2024-04-01,13101,2024,4,88250


In [6]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_agrupado['Viajes Ocasionales'])
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

ADF Statistic: -2.6631908036096936
p-value: 0.08062730698013165


In [8]:
import pmdarima as pm
import matplotlib.pyplot as plt

# Definir la serie temporal (asegúrate de tener la columna 'Viajes Ocasionales' y un índice de fecha)
series = df_agrupado['Viajes Ocasionales']

# Aplicar auto_arima para encontrar el mejor modelo SARIMA
model = pm.auto_arima(series,
                      seasonal=True,  # Estacionalidad (debe ser True para SARIMA)
                      m=12,  # Periodicidad estacional, 12 para datos mensuales
                      stepwise=True,  # Búsqueda paso a paso para eficiencia
                      trace=True,  # Para ver el progreso de la búsqueda de parámetros
                      suppress_warnings=True,  # Suprimir advertencias
                      max_order=10)  # Máximos órdenes para p, q, P, Q

# Ver el resumen del modelo seleccionado
print(model.summary())

# Hacer predicciones para los próximos 12 meses
forecast = model.predict(n_periods=12)

# Visualización de las predicciones
plt.figure(figsize=(10, 5))
plt.plot(series, label='Histórico')
plt.plot(range(len(series), len(series) + 12), forecast, label='Predicción', color='red')
plt.legend()
plt.show()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# Ajusta el modelo SARIMA
# (p, d, q) son los parámetros no estacionales, y (P, D, Q, S) los estacionales
model = SARIMAX(df_agrupado['Viajes Ocasionales'],
                order=(p, d, q),  # Parámetros no estacionales
                seasonal_order=(P, D, Q, S),  # Parámetros estacionales
                enforce_stationarity=False,  # No forzar estacionariedad
                enforce_invertibility=False)  # No forzar invertibilidad

model_fit = model.fit(disp=False)

# Ver el resumen del modelo
print(model_fit.summary())


NameError: name 'p' is not defined